In [1]:
import import_ipynb
import Group7_origin as groupA 
###import group9 as groupB
import numpy as np
import pygame
import time
#import test1

importing Jupyter notebook from Group7_origin.ipynb
pygame 2.1.0 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#competition_list=[g3,g5,g12]

In [3]:
def update_by_man(event, mat):
    """
    This function detects the mouse click on the game window. Update the state matrix of the game.
    input:
        event:pygame event, which are either quit or mouse click)
        mat: 2D matrix represents the state of the game
    output:
        mat: updated matrix
    """
    done = False
    if event.type == pygame.QUIT:
        done = True
    if event.type == pygame.MOUSEBUTTONDOWN:
        (x, y) = event.pos
        row = round((y - 40) / 40)
        col = round((x - 40) / 40)
        mat[row][col] = 1
    return mat, done


# draw the board
def draw_board(screen):
    """
    This function draws the board with lines.
    input: game windows
    output: none
    """
    d = int(560 / (M - 1))
    black_color = [0, 0, 0]
    board_color = [241, 196, 15]
    screen.fill(board_color)
    for h in range(0, M):
        pygame.draw.line(screen, black_color, [40, h * d + 40], [600, 40 + h * d], 1)
        pygame.draw.line(screen, black_color, [40 + d * h, 40], [40 + d * h, 600], 1)


# draw the stone
def draw_stone(screen, mat):
    """
    This functions draws the stones according to the mat. It draws a black circle for matrix element 1(human),
    it draws a white circle for matrix element -1 (computer)
    input:
        screen: game window, onto which the stones are drawn
        mat: 2D matrix representing the game state
    output:
        none
    """
    black_color = [0, 0, 0]
    white_color = [255, 255, 255]
    M = len(mat)
    d = int(560 / (M - 1))
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            if mat[i][j] == 1:
                pos = [40 + d * j, 40 + d * i]
                pygame.draw.circle(screen, black_color, pos, 18, 0)
            elif mat[i][j] == -1:
                pos = [40 + d * j, 40 + d * i]
                pygame.draw.circle(screen, white_color, pos, 18, 0)
    pygame.display.update()
    
    

def check_for_done(mat):
    result = game_result(mat)
    if result:
        return True, result
    else:
        if len(np.where(mat == 0)[0]) == 0:
            return True, 0.5
        else:
            return False, 0


def game_result(board):
    # check if game is over
    rowcum = np.cumsum(board, 0)
    colcum = np.cumsum(board, 1)

    rowsum = rowcum[4:, :] - np.vstack((np.zeros(M), rowcum[:M - 5, :]))
    colsum = colcum[:, 4:] - np.hstack((np.zeros((M, 1)), colcum[:, :M - 5]))
    diag_tl = np.array([
        board[i:i + 5, j:j + 5]
        for i in range(M - 4)
        for j in range(M - 4)
    ])
    diag_sum_tl = diag_tl.trace(axis1=1, axis2=2)
    diag_sum_tr = diag_tl[:, ::-1].trace(axis1=1, axis2=2)

    player_one_wins = 5 in rowsum
    player_one_wins += 5 in colsum
    player_one_wins += 5 in diag_sum_tl
    player_one_wins += 5 in diag_sum_tr

    if player_one_wins:
        return 1

    player_two_wins = -5 in rowsum
    player_two_wins += -5 in colsum
    player_two_wins += -5 in diag_sum_tl
    player_two_wins += -5 in diag_sum_tr

    if player_two_wins:
        return -1

    if np.all(board != 0):
        return 0.

    # if not over - no result
    return None

# render the GUI
def render(screen, mat):
    """
    Draw the updated game with lines and stones using function draw_board and draw_stone
    input:
        screen: game window, onto which the stones are drawn
        mat: 2D matrix representing the game state
    output:
        none
    """
    draw_board(screen)
    draw_stone(screen, mat)
    pygame.display.update()

# main function

In [4]:
import pygame
import time
import random
import copy
import numpy as np

def main():     
    board = groupA.Board()
    board.init_board()
    ai = groupA.MCTS()  
    
    global M
    M=8
    pygame.init()
    screen = pygame.display.set_mode((80*M, 80*M))
    pygame.display.set_caption('Interface of Five-in-a-Row for A first')
    done = False
    pc_step = 0    
    mat=np.zeros((M,M),int)   
    #mat[int(M/2)][int(M/2)]=1
#     print(groupA)
#     print(groupB)
    draw_board(screen)
    pygame.display.update()
#     if order==2:
#         groupA,groupB=groupB,groupA
    mat1,mat2=np.zeros((M,M),int),np.zeros((M,M),int)  
    while not done:
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                done=True
                pygame.quit()
                ###it is group A turn
            draw_stone(screen, mat)
            #print("good start")
            if event.type!=pygame.QUIT:
            #if event.type==pygame.ACTIVEEVENT:
                start_time_for_white_player=time.time()
                pc_step+=1
                print(pc_step,'pc step')
                
                a = list(np.reshape(mat, (1,64)).astype(int)[0])
                pos_a = np.where(np.array(a) == 0)
                if len(pos_a)!=M**2:
                    pre_move = list(set(board.availables)-set(pos_a))[0]
                    board.update(-1,pre_move)
                
                move = ai.take_action(board)
                board.update(1, move)
                board_list=list(board.states.values())
                mat1=np.array(board_list).reshape(M,M)
                
                end_time_for_white_player=time.time()

                ##if time is too long give a random pos instead###
                if end_time_for_white_player-start_time_for_white_player>5.1:
                    availabel_place=np.where(mat==0)
                    select_pos=random.randint(0,len(availabel_place[0])-1)
                    mat[availabel_place[0][select_pos],availabel_place[1][select_pos]]=1
                    print("Black exceeds the time limit and got a random stone")

                else:
                    mat=mat1
                    print("Black finished the stone")

                ##############    

                draw_stone(screen, mat)
                print("mat=\n",mat)
                done, _ = check_for_done(mat)
                if done:
                    print('winer is:',_)
                    break
                ### it is group B turn


                
                availabel_place=np.where(mat==0)
                select_pos=random.randint(0,len(availabel_place[0])-1)
                mat[availabel_place[0][select_pos],availabel_place[1][select_pos]]=-1
                
                

                print("White finished the stone")
                ##############                 

                done, _ = check_for_done(mat)
                print('now mat is\n',mat)                
                # check for win or tie
                if done==True:
                    print('winer is:',_)
                    break
        
        if game_result(mat) is not None:
            myfont = pygame.font.Font(None, 40)
            print('finished')
            text = ""
            if game_result(mat) == 1:
                text = "Black player wins!"
            elif game_result(mat) == -1:
                text = "White player wins!"
            else:
                text = "Nobody wins!"
            textImage = myfont.render(text, True, (255, 255, 255))
            screen.blit(textImage, (int(M*80/2)-len(text)*8, int(M*80/2)-20))
            print("#################end#######################")
            pygame.display.update()
            time.sleep(10)
            pygame.quit()
            


In [5]:
if __name__ == '__main__':
    main()

1 pc step
Black finished the stone
mat=
 [[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
White finished the stone
now mat is
 [[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0 -1  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
2 pc step
Black finished the stone
mat=
 [[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
White finished the stone
now mat is
 [[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
3 pc step
Black finished the stone
mat=
 [[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 0]
 [0 0 

In [16]:
import numpy as np
a = np.array([[1,4], [2, 3]])
print(a)

[[1 4]
 [2 3]]


In [17]:
a.reshape(1,2**2).tolist()[0]

[1, 4, 2, 3]

In [34]:
mat = np.zeros((8,8))
# mat=mat.tolist()
a = list(np.reshape(mat, (1,64)).astype(int)[0])

In [41]:
# l1 = [1, 5, 1, 8, 9, 15, 6, 2, 1]?
mdasdasdas
a = 0

NameError: name 'mdasdasdas' is not defined